# 패널 검색 시스템 (PostgreSQL + pgvector)

## 시스템 구조

### 데이터베이스 구조

**testcl 스키마:**
- `panel_embeddings`: 임베딩 벡터 저장
  - mb_sn (고유번호)
  - embedding (768차원 벡터)
  - 기타 메타데이터

**RawData 스키마:**
- `welcome_1st`: 기본정보 (mb_sn, gender, birth_year, location, detail_location)
- `welcome_2nd`: 상세정보 (id, mb_sn, data-JSON)
- `quick_answer`: 설문답변 (mb_sn, answers, gender, age, location)
- `quick_question`: 설문문항 (origin_qid, qid, question)

### 검색 프로세스
```
자연어 쿼리
    ↓
1. 쿼리 파싱 (LLM)
    ↓
2. 텍스트화
    ↓
3. 임베딩 (768차원)
    ↓
4. pgvector 유사도 검색 (Top-10)
    ↓
5. RawData 테이블 조인
    ↓
6. 전체 데이터 출력
```

## 1. 환경 설정

In [132]:
import os
import json
import numpy as np
import psycopg2
import pandas as pd
from typing import Dict, List, Any, Tuple
from dataclasses import dataclass
from datetime import datetime

# LangChain
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic

# Embeddings
from sentence_transformers import SentenceTransformer

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


## 2. 설정

In [133]:
@dataclass
class Config:
    """시스템 설정"""
    
    # LLM
    llm_model: str = "claude-3-5-haiku-20241022"
    llm_temperature: float = 0.0
    
    # 임베딩 (768차원)
    embedding_model: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    embedding_dim: int = 768
    
    # PostgreSQL
    db_host: str = "localhost"
    db_port: int = 5432
    db_name: str = "ProjectDB"
    db_user: str = "postgres"
    db_password: str = "3961"
    
    # 스키마 및 테이블
    embedding_schema: str = "testcl"
    embedding_table: str = "panel_embeddings"
    rawdata_schema: str = "RawData"
    
    # 검색
    top_k: int = 10

config = Config()
print("✅ 설정 완료")

✅ 설정 완료


## 3. 쿼리 파서

In [134]:
class QueryParser:
    """자연어 쿼리 파싱 - 메타데이터 추출"""
    
    def __init__(self, config: Config):
        self.llm = ChatAnthropic(
            model=config.llm_model,
            temperature=config.llm_temperature
        )
        
        self.prompt = PromptTemplate(
            input_variables=["query"],
            template="""당신은 패널 검색 시스템의 쿼리 분석 전문가입니다.
사용자의 자연어 질의를 분석하여 구조화된 메타데이터를 추출하는 것이 당신의 임무입니다.

=== 추출 규칙 ===

1. **location** (지역명)
   - 유효값: 서울, 부산, 대구, 인천, 광주, 대전, 울산, 세종, 경기, 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주
   - 없으면: null

2. **age_min, age_max** (나이 범위)
   - 10대 → 10-19세
   - 20대 → 20-29세
   - 30대 → 30-39세
   - 40대 → 40-49세
   - 50대 → 50-59세
   - 60대 → 60-69세
   - 70대 → 70-79세
   - 80대 → 80-89세
   - "청년" → 20-39세
   - "중년" → 40-59세
   - "노년" → 60-79세
   - 없으면: null

3. **gender** (성별)
   - "남", "여", 또는 null
   - "남성" → "남"
   - "여성" → "여"

4. **keywords** (핵심 특성)
   - 패널의 특성/행동/상태만 포함
   - 예: 흡연, 음주, 운동, 직업, 기혼, 미혼, 학생, 질병명 등
   - ⚠️ 제외사항: 인원 수("5명", "10명"), 결과 개수, 동사("찾아줘", "검색")

=== Few-Shot 예시 ===

예시 1:
입력: "서울에 사는 30대 기혼 남성 5명 찾아줘"
출력:
{{
    "location": "서울",
    "age_min": 30,
    "age_max": 39,
    "gender": "남",
    "keywords": ["기혼"]
}}

예시 2:
입력: "부산 20대 여성 중에 운동 좋아하는 사람"
출력:
{{
    "location": "부산",
    "age_min": 20,
    "age_max": 29,
    "gender": "여",
    "keywords": ["운동"]
}}

예시 3:
입력: "경기도 청년층 흡연자"
출력:
{{
    "location": "경기",
    "age_min": 20,
    "age_max": 39,
    "gender": null,
    "keywords": ["흡연"]
}}

예시 4:
입력: "대전 중년 여성 당뇨병 환자"
출력:
{{
    "location": "대전",
    "age_min": 40,
    "age_max": 59,
    "gender": "여",
    "keywords": ["당뇨병"]
}}

예시 5:
입력: "인천 10대 학생"
출력:
{{
    "location": "인천",
    "age_min": 10,
    "age_max": 19,
    "gender": null,
    "keywords": ["학생"]
}}

=== 분석 과정 (Chain-of-Thought) ===

질의: {query}

단계별로 생각하세요:
1. 지역명이 명시되어 있는가?
2. 나이 또는 연령대 표현이 있는가?
3. 성별이 명시되어 있는가?
4. 패널의 특성을 나타내는 키워드는 무엇인가?
5. 인원 수나 결과 개수는 제외했는가?

=== 최종 출력 ===

위 분석을 바탕으로 JSON 형식으로만 반환하세요:
{{
    "location": "지역명" 또는 null,
    "age_min": 숫자 또는 null,
    "age_max": 숫자 또는 null,
    "gender": "남" 또는 "여" 또는 null,
    "keywords": ["키워드1", "키워드2"] 또는 []
}}

JSON만 반환하세요. 추가 설명은 불필요합니다."""
        )
    
    def parse(self, query: str) -> Dict[str, Any]:
        """쿼리 파싱 - 메타데이터만 추출"""
        prompt_text = self.prompt.format(query=query)
        response = self.llm.invoke(prompt_text)
        
        response_text = response.content
        
        try:
            if '```json' in response_text:
                json_text = response_text.split('```json')[1].split('```')[0].strip()
            elif '```' in response_text:
                json_text = response_text.split('```')[1].strip()
            else:
                json_text = response_text
            
            result = json.loads(json_text)
            
            # raw_query가 없으면 추가
            if 'raw_query' not in result:
                result['raw_query'] = query
                
            return result
        except Exception as e:
            print(f"⚠️ 파싱 실패: {e}")
            return {
                "location": None,
                "age_min": None,
                "age_max": None,
                "gender": None,
                "keywords": []
            }

print("✅ QueryParser 정의 완료")

✅ QueryParser 정의 완료


## 4. 텍스트화

In [135]:
class QueryTextualizer:
    """메타데이터를 검색용 한 문장으로 변환 - 프롬프트 엔지니어링"""
    
    def __init__(self, config: Config):
        self.llm = ChatAnthropic(
            model=config.llm_model,
            temperature=config.llm_temperature
        )
        
        self.prompt = PromptTemplate(
            input_variables=["location", "age_min", "age_max", "gender", "keywords"],
            template="""당신은 검색 엔진 최적화(SEO) 전문가이자 임베딩 검색 시스템 전문가입니다.
주어진 메타데이터를 벡터 임베딩에 최적화된 자연스러운 한국어 문장으로 변환하는 것이 당신의 임무입니다.

=== 입력 메타데이터 ===
- 지역: {location}
- 나이 범위: {age_min}세 ~ {age_max}세
- 성별: {gender}
- 키워드: {keywords}

=== 작성 원칙 ===

1. **간결성**: 한 문장으로 핵심만 표현
2. **자연스러움**: 구어체가 아닌 명확한 서술형
3. **키워드 밀도**: 검색 관련 핵심 단어를 앞쪽에 배치
4. **정보 완결성**: 모든 메타데이터를 누락 없이 포함
5. **불필요한 요소 제거**: 조사, 수식어, 인원 수 절대 금지

=== 표현 가이드 ===

**지역 표현:**
- "{location} 거주" 또는 "{location} 지역"
- "지역 무관"인 경우 생략 가능

**나이 표현:**
- 10대, 20대, 30대 등 연령대 표현 우선
- 정확한 범위: "{age_min}세부터 {age_max}세"
- "나이 무관"인 경우 생략

**성별 표현:**
- "남성", "여성" (줄임말 "남", "여" 대신 정식 표현)
- "성별 무관"인 경우 생략

**키워드 표현:**
- 명사형으로 간결하게
- 예: "흡연자", "운동 애호가", "기혼자", "학생"

=== Few-Shot 예시 ===

예시 1:
입력: 지역=경기, 나이=20-29, 성별=남, 키워드=[흡연]
출력: 경기 거주 20대 남성 흡연자

예시 2:
입력: 지역=서울, 나이=30-39, 성별=여, 키워드=[기혼, 운동]
출력: 서울 지역 30대 기혼 여성 운동 애호가

예시 3:
입력: 지역=부산, 나이=40-59, 성별=남, 키워드=[당뇨병]
출력: 부산 거주 40대 50대 남성 당뇨병 환자

예시 4:
입력: 지역=대전, 나이=무관, 성별=여, 키워드=[직장인]
출력: 대전 지역 여성 직장인

예시 5:
입력: 지역=무관, 나이=20-29, 성별=무관, 키워드=[학생]
출력: 20대 학생

예시 6:
입력: 지역=제주, 나이=60-69, 성별=여, 키워드=[]
출력: 제주 거주 60대 여성

예시 7:
입력: 지역=인천, 나이=10-19, 성별=남, 키워드=[게임, 학생]
출력: 인천 지역 10대 남학생 게임 선호

=== 작성 과정 (Chain-of-Thought) ===

단계별로 생각하세요:
1. 가장 중요한 정보는 무엇인가? (지역 > 나이 > 성별 > 키워드 순)
2. 어떤 정보가 있고 없는가?
3. 키워드를 자연스럽게 연결하는 방법은?
4. 임베딩 검색에서 유사도를 높일 핵심 단어는?

=== 최종 출력 ===

위 원칙과 예시를 참고하여, 검색 임베딩에 최적화된 한 문장을 생성하세요.

⚠️ 주의:
- 인원 수("5명", "10명" 등) 절대 포함 금지
- 결과 개수 표현 금지
- 동사나 명령어("찾아줘", "검색") 금지
- 오직 패널의 특성만 서술

한 문장만 반환하세요."""
        )
    
    def textualize(self, parsed_query: Dict[str, Any]) -> str:
        """메타데이터 → 한 문장 텍스트 변환 (인원 수 제외)"""
        
        # 메타데이터 추출
        location = parsed_query.get('location') or '지역 무관'
        age_min = parsed_query.get('age_min') or '나이 무관'
        age_max = parsed_query.get('age_max') or '나이 무관'
        gender = parsed_query.get('gender') or '성별 무관'
        keywords = parsed_query.get('keywords', [])
        
        # 키워드에서 인원 수 관련 표현 제거
        filtered_keywords = [
            kw for kw in keywords 
            if not any(num in kw for num in ['명', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])
        ]
        keywords_str = ', '.join(filtered_keywords) if filtered_keywords else '키워드 없음'
        
        # LLM으로 한 문장 생성
        prompt_text = self.prompt.format(
            location=location,
            age_min=age_min,
            age_max=age_max,
            gender=gender,
            keywords=keywords_str
        )
        
        try:
            response = self.llm.invoke(prompt_text)
            search_text = response.content.strip()
            
            # 따옴표나 불필요한 기호 제거
            search_text = search_text.strip('"\'')
            
            return search_text
            
        except Exception as e:
            print(f"⚠️ 텍스트화 실패: {e}")
            # 폴백: 간단한 텍스트 조합 (인원 수 제외)
            components = []
            if location != '지역 무관':
                components.append(f"{location} 거주")
            if age_min != '나이 무관' and age_max != '나이 무관':
                components.append(f"{age_min}세에서 {age_max}세")
            if gender != '성별 무관':
                components.append(gender)
            if filtered_keywords:
                components.extend(filtered_keywords)
            return ' '.join(components)

print("✅ QueryTextualizer 정의 완료")

✅ QueryTextualizer 정의 완료


## 5. 임베딩 생성기

In [136]:
class EmbeddingGenerator:
    """768차원 임베딩 생성"""
    
    def __init__(self, model_name: str):
        print(f"임베딩 모델 로딩: {model_name}")
        self.model = SentenceTransformer(model_name)
        dim = self.model.get_sentence_embedding_dimension()
        print(f"✅ 로딩 완료 (차원: {dim})")
    
    def embed(self, text: str) -> np.ndarray:
        """텍스트 → 벡터"""
        return self.model.encode(text, convert_to_numpy=True)

print("✅ EmbeddingGenerator 정의 완료")

✅ EmbeddingGenerator 정의 완료


## 6. 데이터베이스 검색기

In [137]:
class DatabaseSearcher:
    """PostgreSQL 검색 및 데이터 조인"""
    
    def __init__(self, config: Config):
        self.config = config
        self.conn = None
        self.connect()
    
    def connect(self):
        """DB 연결"""
        try:
            self.conn = psycopg2.connect(
                host=self.config.db_host,
                port=self.config.db_port,
                dbname=self.config.db_name,
                user=self.config.db_user,
                password=self.config.db_password
            )
            self.conn.autocommit = True  # ⭐ 추가
            print(f"✅ DB 연결: {self.config.db_name}")
        except Exception as e:
            print(f"❌ DB 연결 실패: {e}")
            raise
    
    def search_similar(self, query_embedding: np.ndarray, top_k: int) -> List[str]:
        """벡터 유사도 검색
        
        Returns:
            List[mb_sn]: Top-K 고유번호 리스트
        """
        cursor = self.conn.cursor()
        
        # 벡터를 문자열로 변환
        vector_str = '[' + ','.join(map(str, query_embedding)) + ']'
        
        # 코사인 유사도 검색
        query = f"""
        SELECT 
            mb_sn,
            1 - (embedding <=> '{vector_str}'::vector) as similarity
        FROM "{self.config.embedding_schema}"."{self.config.embedding_table}"
        ORDER BY similarity DESC
        LIMIT {top_k}
        """
        
        try:
            print("\n🔍 벡터 유사도 검색 중...")
            cursor.execute(query)
            results = cursor.fetchall()
            
            mb_sn_list = [row[0] for row in results]
            
            print(f"✅ {len(mb_sn_list)}개 패널 발견")
            for i, (mb_sn, sim) in enumerate(results, 1):
                print(f"  {i}. {mb_sn} (유사도: {sim:.4f})")
            
            cursor.close()
            return mb_sn_list
            
        except Exception as e:
            print(f"❌ 검색 실패: {e}")
            cursor.close()
            raise
    
    def get_panel_data(self, mb_sn_list: List[str]) -> pd.DataFrame:
        """패널 전체 데이터 조회
        
        Args:
            mb_sn_list: 조회할 고유번호 리스트
            
        Returns:
            DataFrame: 모든 테이블 조인된 결과
        """
        cursor = self.conn.cursor()
        
        # mb_sn 리스트를 SQL IN 절에 사용
        mb_sn_tuple = "('" + "','" .join(mb_sn_list) + "')"
        
        # 모든 테이블 조인 쿼리
        query = f"""
        SELECT 
            w1.mb_sn,
            w1.gender as w1_gender,
            w1.birth_year,
            w1.location as w1_location,
            w1.detail_location,
            w2.data as welcome_2nd_data,
            qa.answers as quick_answers,
            qa.gender as qa_gender,
            qa.age as qa_age,
            qa.location as qa_location
        FROM "{self.config.rawdata_schema}"."welcome_1st" w1
        LEFT JOIN "{self.config.rawdata_schema}"."welcome_2nd" w2 ON w1.mb_sn = w2.mb_sn
        LEFT JOIN "{self.config.rawdata_schema}"."quick_answer" qa ON w1.mb_sn = qa.mb_sn
        WHERE w1.mb_sn IN {mb_sn_tuple}
        ORDER BY ARRAY_POSITION(ARRAY{mb_sn_list}::text[], w1.mb_sn)
        """
        
        try:
            print("\n📊 패널 전체 데이터 조회 중...")
            cursor.execute(query)
            results = cursor.fetchall()
            
            # DataFrame 생성
            columns = [
                'mb_sn', 'gender', 'birth_year', 'location', 'detail_location',
                'welcome_2nd_data', 'quick_answers', 
                'qa_gender', 'qa_age', 'qa_location'
            ]
            
            df = pd.DataFrame(results, columns=columns)
            
            print(f"✅ {len(df)}개 패널 데이터 조회 완료")
            
            cursor.close()
            return df
            
        except Exception as e:
            print(f"❌ 데이터 조회 실패: {e}")
            cursor.close()
            raise
    
    def get_question_list(self) -> pd.DataFrame:
        """설문 문항 목록 조회"""
        cursor = self.conn.cursor()
        
        query = f"""
        SELECT origin_qid, qid, question
        FROM "{self.config.rawdata_schema}"."quick_question"
        ORDER BY qid
        """
        
        try:
            cursor.execute(query)
            results = cursor.fetchall()
            
            df = pd.DataFrame(results, columns=['origin_qid', 'qid', 'question'])
            
            cursor.close()
            return df
            
        except Exception as e:
            print(f"❌ 문항 조회 실패: {e}")
            cursor.close()
            raise
    
    def close(self):
        """연결 종료"""
        if self.conn:
            self.conn.close()
            print("✅ DB 연결 종료")

print("✅ DatabaseSearcher 정의 완료")

✅ DatabaseSearcher 정의 완료


## 7. 통합 검색 시스템

In [138]:
class PanelSearchSystem:
    """통합 패널 검색 시스템"""
    
    def __init__(self, config: Config):
        self.config = config
        
        print("\n🚀 시스템 초기화 중...\n")
        
        self.parser = QueryParser(config)
        self.textualizer = QueryTextualizer(config)
        self.embedder = EmbeddingGenerator(config.embedding_model)
        self.searcher = DatabaseSearcher(config)
        
        print("\n✅ 시스템 초기화 완료!\n")
    
    def search(self, query: str, top_k: int = None) -> Dict[str, Any]:
        """통합 검색"""
        print(f"\n{'='*80}")
        print(f"🔎 검색 쿼리: {query}")
        print(f"{'='*80}\n")
        
        # 1. 쿼리 파싱 (메타데이터 추출)
        print("[1단계] 쿼리 파싱 - 메타데이터 추출")
        parsed = self.parser.parse(query)
        print(f"  ✓ 지역: {parsed.get('location')}")
        print(f"  ✓ 나이: {parsed.get('age_min')}~{parsed.get('age_max')}세")
        print(f"  ✓ 성별: {parsed.get('gender')}")
        print(f"  ✓ 키워드: {parsed.get('keywords')}")
        
        # 2. 텍스트화 (프롬프트 엔지니어링)
        print("\n[2단계] 텍스트화 - 한 문장 생성 (프롬프트 엔지니어링)")
        search_text = self.textualizer.textualize(parsed)
        print(f"  ✓ 검색 텍스트: {search_text}")
        
        # 3. 임베딩
        print("\n[3단계] 임베딩 생성")
        embedding = self.embedder.embed(search_text)
        print(f"  ✓ 차원: {embedding.shape}")
        
        # 4. 벡터 검색
        print("\n[4단계] 벡터 유사도 검색")
        top_k = top_k or self.config.top_k
        mb_sn_list = self.searcher.search_similar(embedding, top_k)
        
        # 5. 전체 데이터 조회
        print("\n[5단계] 전체 데이터 조회")
        panel_data = self.searcher.get_panel_data(mb_sn_list)
        
        print(f"\n{'='*80}")
        print(f"✅ 검색 완료!")
        print(f"{'='*80}\n")
        
        return {
            'query': query,
            'parsed_query': parsed,
            'search_text': search_text,
            'mb_sn_list': mb_sn_list,
            'panel_data': panel_data,
            'total_count': len(panel_data)
        }
    
    def close(self):
        """시스템 종료"""
        self.searcher.close()

print("✅ PanelSearchSystem 정의 완료")

✅ PanelSearchSystem 정의 완료


## 8. 시스템 초기화

In [ ]:
# API 키 설정
os.environ['ANTHROPIC_API_KEY'] = 'api_key'
# 설정
config = Config(
    embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    db_host="localhost",
    db_port=5432,
    db_name="ProjectDB",
    db_user="postgres",
    db_password="3961",
    embedding_schema="testcl",
    embedding_table="panel_embeddings",
    rawdata_schema="RawData",
    top_k=10
)

# 시스템 초기화
search_system = PanelSearchSystem(config)


🚀 시스템 초기화 중...

임베딩 모델 로딩: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
✅ 로딩 완료 (차원: 768)
✅ DB 연결: ProjectDB

✅ 시스템 초기화 완료!



## 9. 검색 테스트

In [140]:
# 테스트 쿼리
query = "서울에 사는 기혼 남자 5명"

results = search_system.search(query, top_k=5)


🔎 검색 쿼리: 서울에 사는 기혼 남자 5명

[1단계] 쿼리 파싱 - 메타데이터 추출
  ✓ 지역: 서울
  ✓ 나이: None~None세
  ✓ 성별: 남
  ✓ 키워드: ['기혼']

[2단계] 텍스트화 - 한 문장 생성 (프롬프트 엔지니어링)
  ✓ 검색 텍스트: 서울 거주 기혼 남성

[3단계] 임베딩 생성
  ✓ 차원: (768,)

[4단계] 벡터 유사도 검색

🔍 벡터 유사도 검색 중...
✅ 5개 패널 발견
  1. w227779759538 (유사도: 0.0631)
  2. w376313529907746 (유사도: 0.0607)
  3. w264897589315675 (유사도: 0.0550)
  4. w272029033675954 (유사도: 0.0548)
  5. w7462586643341 (유사도: 0.0542)

[5단계] 전체 데이터 조회

📊 패널 전체 데이터 조회 중...
✅ 3개 패널 데이터 조회 완료

✅ 검색 완료!



## 10. 결과 출력 - 요약

In [141]:
# 기본 정보 출력
print("\n" + "="*80)
print("📊 검색 결과 요약")
print("="*80)

print(f"\n🔎 검색 쿼리: {results['query']}")
print(f"📝 검색 텍스트: {results['search_text']}")
print(f"\n📋 총 {results['total_count']}명의 패널 발견\n")

# DataFrame 출력
df = results['panel_data']

# 기본 정보만 출력 (상위 5개)
print("-"*80)
print("기본 정보 (Welcome 1st) - 상위 5개")
print("-"*80)
display(df[['mb_sn', 'gender', 'birth_year', 'location', 'detail_location']].head(5))


📊 검색 결과 요약

🔎 검색 쿼리: 서울에 사는 기혼 남자 5명
📝 검색 텍스트: 서울 거주 기혼 남성

📋 총 3명의 패널 발견

--------------------------------------------------------------------------------
기본 정보 (Welcome 1st) - 상위 5개
--------------------------------------------------------------------------------


,mb_sn,gender,birth_year,location,detail_location
0,w376313529907746,여성,2005,서울,노원구
1,w264897589315675,여성,2004,서울,관악구
2,w272029033675954,여성,2003,서울,동대문구


## 11. 결과 출력 - Welcome 2nd 상세 정보

In [142]:
# Welcome 2nd 데이터 파싱 및 출력
print("\n" + "="*80)
print("📊 Welcome 2nd 상세 정보 - 상위 5개")
print("="*80 + "\n")

for idx, row in df.head(5).iterrows():
    mb_sn = row['mb_sn']
    welcome_2nd_data = row['welcome_2nd_data']
    
    print(f"\n{'='*80}")
    print(f"패널 {idx + 1}: {mb_sn}")
    print("="*80)
    
    if welcome_2nd_data:
        try:
            # JSON 파싱
            if isinstance(welcome_2nd_data, str):
                data = json.loads(welcome_2nd_data)
            else:
                data = welcome_2nd_data
            
            # 주요 정보 출력
            print("\n[Welcome 2nd 데이터]")
            for key, value in data.items():
                print(f"  {key}: {value}")
                
        except Exception as e:
            print(f"  ⚠️ JSON 파싱 실패: {e}")
    else:
        print("  ℹ️ Welcome 2nd 데이터 없음")


📊 Welcome 2nd 상세 정보 - 상위 5개


패널 1: w376313529907746

[Welcome 2nd 데이터]
  mb_sn: w376313529907746
  직무: None
  직업: 대학생/대학원생
  가족수: 4명
  자녀수: 0
  결혼여부: 미혼
  최종학력: 대학교 재학(휴학 포함)
  흡연경험: None
  보유전제품: ['TV', '냉장고', '김치냉장고', '세탁기', '에어컨', '제습기', '공기청정기(에어 케어)', '정수기', '일반청소기', '로봇청소기', '무선청소기(예:다이슨, 코드제로, 제트 등)', '비데', '건조기', '전기레인지(하이라이트, 인덕션, 핫플레이트 등)', '에어프라이기', '노트북', '태블릿PC (아이패드, 갤럭시 탭 등)', '데스크탑', '무선 이어폰(예: 에어팟, 갤럭시 버즈 등)', '인공지능 AI 스피커', '블루투스 스피커']
  음용경험 술: None
  자동차 모델: None
  보유차량여부: 없다
  자동차 제조사: None
  월평균 가구소득: 월 500~599만원
  월평균 개인소득: 월 100~199만원
  보유 휴대폰 모델명: 아이폰 12/ 12mini
  흡연경험 담배브랜드: None
  음용경험 술(기타내용): None
  보유 휴대폰 단말기 브랜드: 애플 (아이폰)
  흡연경험 담배 브랜드(기타내용): None
  흡연경험 담배브랜드(기타브랜드): None
  궐련형 전자담배/가열식 전자담배 이용경험: None

패널 2: w264897589315675

[Welcome 2nd 데이터]
  mb_sn: w264897589315675
  직무: 공부
  직업: 재수생
  가족수: 2명
  자녀수: 0
  결혼여부: 미혼
  최종학력: 고등학교 졸업 이하
  흡연경험: ['일반 담배']
  보유전제품: ['TV', '냉장고', '김치냉장고', '세탁기', '에어컨', '일반청소기', '무선청소기(예:다이슨, 코드제로, 제트 등)', '커피 머신(에스프레소 머신,

## 12. 결과 출력 - Quickpoll 답변

In [143]:
# Quickpoll 답변 출력
print("\n" + "="*80)
print("📊 Quickpoll 답변 - 상위 5개")
print("="*80 + "\n")

# 설문 문항 목록 조회
questions_df = search_system.searcher.get_question_list()
question_dict = dict(zip(questions_df['qid'], questions_df['question']))

for idx, row in df.head(5).iterrows():
    mb_sn = row['mb_sn']
    answers = row['quick_answers']
    
    print(f"\n{'='*80}")
    print(f"패널 {idx + 1}: {mb_sn}")
    print(f"성별: {row['qa_gender']}, 나이: {row['qa_age']}, 지역: {row['qa_location']}")
    print("="*80)
    
    if answers:
        try:
            # JSON 파싱
            if isinstance(answers, str):
                answer_dict = json.loads(answers)
            else:
                answer_dict = answers
            
            print("\n[설문 답변]")
            for qid, answer in answer_dict.items():
                question = question_dict.get(qid, "문항 정보 없음")
                print(f"\n  {qid}: {question}")
                print(f"  답변: {answer}")
                
        except Exception as e:
            print(f"  ⚠️ 답변 파싱 실패: {e}")
    else:
        print("  ℹ️ 설문 답변 없음")
    
    print("\n" + "-"*80)


📊 Quickpoll 답변 - 상위 5개


패널 1: w376313529907746
성별: 여, 나이: 20, 지역: 서울

[설문 답변]

  Q010: 여러분이 현재 이용 중인 OTT 서비스는 몇 개인가요?
  답변: 1개

  Q035: 여러분은 평소 체력 관리를 위해 어떤 활동을 하고 계신가요? 모두 선택해주세요.
  답변: 달리기/걷기, 자전거 타기

--------------------------------------------------------------------------------

패널 2: w264897589315675
성별: 여, 나이: 21, 지역: 서울

[설문 답변]

  Q016: 여러분은 본인을 위해 소비하는 것 중 가장 기분 좋아지는 소비는 무엇인가요?
  답변: 옷/패션관련 제품 구매하기

  Q017: 여러분은 요즘 가장 많이 사용하는 앱은 무엇인가요?
  답변: 동영상 스트리밍 앱 (유튜브, 넷플릭스 등)

--------------------------------------------------------------------------------

패널 3: w272029033675954
성별: 여, 나이: 22, 지역: 서울

[설문 답변]

  Q001: 다음 중 가장 스트레스를 많이 느끼는 상황은 무엇인가요?
  답변: 건강 문제

  Q002: 스트레스를 해소하는 방법으로 주로 사용하는 것은 무엇인가요?
  답변: 수면

  Q003: 현재 본인의 피부 상태에 얼마나 만족하시나요?
  답변: 보통이다

  Q004: 한 달 기준으로 스킨케어 제품에 평균적으로 얼마나 소비하시나요?
  답변: 3만원 이상 ~ 5만원 미만

  Q005: 스킨케어 제품을 구매할 때 가장 중요하게 고려하는 요소는 무엇인가요?
  답변: 성분 및 효과

  Q006: 여러분이 사용해 본 AI 챗봇 서비스는 무엇인가요? 모두 선택해주세요.
  답변: ChatGPT

  Q007: 사용해 본 AI 챗봇 서비스 중 주로 사용하는 것은

## 13. 결과 출력 - 전체 데이터 (JSON)

In [117]:
# 전체 데이터를 JSON 형식으로 출력
print("\n" + "="*80)
print("📊 전체 데이터 (JSON 형식)")
print("="*80 + "\n")

all_panels = []

for idx, row in df.iterrows():
    panel_info = {
        'rank': idx + 1,
        'mb_sn': row['mb_sn'],
        'welcome_1st': {
            'gender': row['gender'],
            'birth_year': row['birth_year'],
            'location': row['location'],
            'detail_location': row['detail_location']
        },
        'welcome_2nd': None,
        'quick_poll': {
            'gender': row['qa_gender'],
            'age': int(row['qa_age']) if row['qa_age'] else None,
            'location': row['qa_location'],
            'answers': None
        }
    }
    
    # Welcome 2nd 파싱
    if row['welcome_2nd_data']:
        try:
            if isinstance(row['welcome_2nd_data'], str):
                panel_info['welcome_2nd'] = json.loads(row['welcome_2nd_data'])
            else:
                panel_info['welcome_2nd'] = row['welcome_2nd_data']
        except:
            pass
    
    # Quick answers 파싱
    if row['quick_answers']:
        try:
            if isinstance(row['quick_answers'], str):
                panel_info['quick_poll']['answers'] = json.loads(row['quick_answers'])
            else:
                panel_info['quick_poll']['answers'] = row['quick_answers']
        except:
            pass
    
    all_panels.append(panel_info)

# JSON 출력
print(json.dumps(all_panels, ensure_ascii=False, indent=2))


📊 전체 데이터 (JSON 형식)

[
  {
    "rank": 1,
    "mb_sn": "w376313529907746",
    "welcome_1st": {
      "gender": "여성",
      "birth_year": "2005",
      "location": "서울",
      "detail_location": "노원구"
    },
    "welcome_2nd": {
      "mb_sn": "w376313529907746",
      "직무": null,
      "직업": "대학생/대학원생",
      "가족수": "4명",
      "자녀수": 0,
      "결혼여부": "미혼",
      "최종학력": "대학교 재학(휴학 포함)",
      "흡연경험": null,
      "보유전제품": [
        "TV",
        "냉장고",
        "김치냉장고",
        "세탁기",
        "에어컨",
        "제습기",
        "공기청정기(에어 케어)",
        "정수기",
        "일반청소기",
        "로봇청소기",
        "무선청소기(예:다이슨, 코드제로, 제트 등)",
        "비데",
        "건조기",
        "전기레인지(하이라이트, 인덕션, 핫플레이트 등)",
        "에어프라이기",
        "노트북",
        "태블릿PC (아이패드, 갤럭시 탭 등)",
        "데스크탑",
        "무선 이어폰(예: 에어팟, 갤럭시 버즈 등)",
        "인공지능 AI 스피커",
        "블루투스 스피커"
      ],
      "음용경험 술": null,
      "자동차 모델": null,
      "보유차량여부": "없다",
      "자동차 제조사": null,
      "월평균 가구소득": "월 500~599만원",
      "월평균 개인

## 14. 결과 저장

In [54]:
# JSON 파일로 저장
output_file = "panel_search_results.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump({
        'query': results['query'],
        'search_text': results['search_text'],
        'total_count': results['total_count'],
        'panels': all_panels
    }, f, ensure_ascii=False, indent=2)

print(f"\n✅ 결과 저장 완료: {output_file}")

# CSV 파일로도 저장 (기본 정보만)
csv_file = "panel_search_results.csv"
df.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"✅ CSV 저장 완료: {csv_file}")


✅ 결과 저장 완료: panel_search_results.json
✅ CSV 저장 완료: panel_search_results.csv


## 15. 시스템 종료

In [ ]:
# 시스템 종료
search_system.close()

## 16. 시스템 요약

### ✅ 구현 완료

1. **쿼리 파싱**: Claude LLM으로 자연어 → 필터
2. **텍스트화**: 메타데이터 → 검색 텍스트
3. **임베딩**: 768차원 벡터 생성
4. **벡터 검색**: pgvector로 Top-10 선정
5. **데이터 조인**: RawData 스키마의 모든 테이블 조인
6. **결과 출력**: 10개 패널의 전체 데이터

### 📊 출력 데이터

각 패널별로:
- **Welcome 1st**: 기본 정보 (성별, 출생년도, 지역, 구)
- **Welcome 2nd**: 상세 정보 (JSON 데이터)
- **Quick Answer**: 설문 답변 (41개 문항)
- **Quick Question**: 설문 문항 매칭

### 🔍 검색 프로세스

```
자연어 쿼리
    ↓
LLM 파싱
    ↓
텍스트화
    ↓
768차원 임베딩
    ↓
pgvector 유사도 검색
    ↓
Top-10 mb_sn
    ↓
RawData 조인
    ↓
전체 데이터 출력
```

### 💾 데이터베이스 구조

**testcl 스키마:**
- `panel_embeddings`: 임베딩 벡터

**RawData 스키마:**
- `welcome_1st`: 기본정보
- `welcome_2nd`: 상세정보 (JSON)
- `quick_answer`: 설문답변
- `quick_question`: 설문문항

### 🚀 사용 예시

```python
# 검색
results = search_system.search("서울에 사는 흡연을 하는 20대 10명")

# 결과 확인
df = results['panel_data']
print(df.head())

# 저장
df.to_csv('results.csv')
```

### 💡 왜 벡터를 문자열로 변환하는가?

**답변: PostgreSQL pgvector의 표준 사용법입니다!**

#### 🔄 처리 흐름

```
Python NumPy 배열 [0.1, 0.2, 0.3, ...]
        ↓ (Python)
문자열 '[0.1, 0.2, 0.3, ...]'
        ↓ (SQL 쿼리 전달)
PostgreSQL 수신
        ↓ (::vector 캐스트)
vector 타입으로 변환 [실제 벡터 타입]
        ↓ (<=> 연산자)
벡터 연산 수행 (코사인 거리 계산)
```

#### ✅ 핵심 포인트

1. **psycopg2 제약**: Python의 PostgreSQL 드라이버는 벡터 타입을 직접 지원하지 않음
2. **문자열 전달**: `'[0.1, 0.2, 0.3]'` 형태로 전달
3. **자동 변환**: PostgreSQL의 `::vector` 캐스트가 문자열을 벡터 타입으로 변환
4. **벡터 연산**: 변환된 벡터끼리 실제 벡터 수학 연산 수행
5. **pgvector 공식 방식**: 이것이 pgvector 공식 문서의 권장 방법

#### 📚 참고: pgvector 연산자

- `<=>`: 코사인 거리 (0 = 동일, 2 = 반대)
- `<->`: L2 거리 (유클리디안 거리)
- `<#>`: 내적의 음수 (inner product)

위 검증 셀을 실행하면 문자열이 벡터로 정상 변환되는 것을 확인할 수 있습니다!

In [ ]:
"""
pgvector의 문자열 변환 작동 원리 검증

PostgreSQL의 pgvector 확장에서 벡터를 문자열로 전달하는 것은 표준 방식입니다.
문자열이 ::vector 캐스트를 통해 실제 벡터 타입으로 변환되어 벡터 연산이 수행됩니다.
"""

import psycopg2
import numpy as np

print("="*80)
print("🔬 pgvector 문자열 → 벡터 변환 검증")
print("="*80)

# 테스트용 벡터 생성 (간단한 3차원 벡터)
test_vector = np.array([0.5, 0.5, 0.5])
vector_str = '[' + ','.join(map(str, test_vector)) + ']'

print(f"\n1. Python NumPy 배열:")
print(f"   타입: {type(test_vector)}")
print(f"   값: {test_vector}")

print(f"\n2. 문자열 변환:")
print(f"   타입: {type(vector_str)}")
print(f"   값: {vector_str}")

# DB 연결
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="ProjectDB",
    user="postgres",
    password="3961"
)

cursor = conn.cursor()

# 3. PostgreSQL에서 문자열이 벡터로 변환되는지 확인
print(f"\n3. PostgreSQL 내부 처리:")

# 테스트 쿼리: 문자열을 vector로 캐스팅하고 타입 확인
cursor.execute(f"""
    SELECT 
        '{vector_str}'::vector as converted_vector,
        pg_typeof('{vector_str}'::vector) as vector_type
""")

result = cursor.fetchone()
print(f"   변환된 값: {result[0]}")
print(f"   PostgreSQL 타입: {result[1]}")

# 4. 실제 유사도 계산 테스트
print(f"\n4. 유사도 연산 테스트:")

# 샘플 청크 1개 가져와서 유사도 계산
cursor.execute(f"""
    SELECT 
        mb_sn,
        chunk_id,
        1 - (embedding <=> '{vector_str}'::vector) as similarity,
        pg_typeof(embedding) as embedding_type
    FROM testcl.panel_chunks
    LIMIT 3
""")

results = cursor.fetchall()
print(f"\n   샘플 유사도 계산 결과:")
for mb_sn, chunk_id, similarity, emb_type in results:
    print(f"   - {mb_sn[:20]}... | Chunk: {chunk_id} | 유사도: {similarity:.4f} | 타입: {emb_type}")

# 5. 벡터 연산이 정상적으로 수행되는지 확인
print(f"\n5. 벡터 연산 확인:")
cursor.execute(f"""
    SELECT 
        '{vector_str}'::vector <=> '{vector_str}'::vector as distance_to_self
""")

distance = cursor.fetchone()[0]
print(f"   동일 벡터 간 거리: {distance}")
print(f"   → {distance} = 0이면 벡터 연산이 정상 작동하는 것입니다 ✅")

cursor.close()
conn.close()

print("\n" + "="*80)
print("✅ 검증 완료: 문자열이 vector 타입으로 변환되어 정상적으로 벡터 연산 수행")
print("="*80)

## 24. pgvector 문자열 변환 작동 원리 검증

In [ ]:
print("""
════════════════════════════════════════════════════════════════════════════════
                        패널 검색 시스템 V2 - 사용법
════════════════════════════════════════════════════════════════════════════════

✅ 구현된 기능
═══════════════════════════════════════════════════════════════════════════════

1. 청킹 기반 검색 (panel_chunks)
   - 하나의 패널이 여러 청크로 분할되어 저장
   - 쿼리 임베딩과 모든 청크 임베딩 비교
   - Top-N 청크 추출 후 패널별 점수 집계
   - 3가지 집계 방식 지원: max, mean, sum

2. 기존 시스템 (panel_embeddings) - 주석 처리
   - 패널당 하나의 통합 임베딩
   - 직접 Top-K 패널 추출

═══════════════════════════════════════════════════════════════════════════════

🔍 검색 프로세스 (Chunk 기반)
═══════════════════════════════════════════════════════════════════════════════

자연어 쿼리
    ↓
[1] 쿼리 파싱 - 메타데이터 추출 (location, age, gender, keywords)
    ↓
[2] 텍스트화 - 프롬프트 엔지니어링으로 한 문장 생성
    ↓
[3] 임베딩 - 768차원 벡터 생성
    ↓
[4] 청크 검색 - Top-N 청크 추출 (예: 100개)
    ↓
[5] 점수 집계 - 패널별 점수 계산 및 Top-K 선정
    ↓
[6] 데이터 조회 - RawData 스키마 조인
    ↓
결과 반환

═══════════════════════════════════════════════════════════════════════════════

📊 집계 방식
═══════════════════════════════════════════════════════════════════════════════

1. max (최고점수)
   - 각 패널의 최고 유사도 청크 점수 사용
   - 특정 속성에 강하게 매칭되는 패널 선호

2. mean (평균점수)
   - 각 패널의 평균 유사도 사용
   - 전반적으로 관련성 높은 패널 선호

3. sum (합계점수)
   - 각 패널의 점수 합계 사용
   - 많은 청크가 매칭되는 패널 선호

═══════════════════════════════════════════════════════════════════════════════

💻 사용 예시
═══════════════════════════════════════════════════════════════════════════════

# 시스템 초기화
search_system = PanelSearchSystemV2(config, use_chunks=True)

# 검색 실행
results = search_system.search(
    query="서울에 사는 20대 여성",
    top_k=10,              # 최종 패널 개수
    top_n_chunks=100,      # 초기 청크 개수
    aggregation='max'      # 'max', 'mean', 'sum'
)

# 결과 확인
print(results['mb_sn_list'])           # 선정된 패널 리스트
print(results['chunk_results'])        # 청크 검색 결과
print(results['panel_data'])           # 패널 전체 데이터

# 종료
search_system.close()

═══════════════════════════════════════════════════════════════════════════════

📁 데이터베이스 구조
═══════════════════════════════════════════════════════════════════════════════

testcl 스키마:
  - panel_embeddings: 패널별 통합 임베딩 (기존 방식)
  - panel_chunks: 패널별 청크 임베딩 (새 방식) ✅ 현재 사용 중

RawData 스키마:
  - welcome_1st: 기본정보
  - welcome_2nd: 상세정보 (JSON)
  - quick_answer: 설문답변
  - quick_question: 설문문항

════════════════════════════════════════════════════════════════════════════════
""")

## 23. 시스템 요약 및 사용법

In [ ]:
"""
# 기존 시스템 테스트 (주석 처리)
# panel_embeddings를 사용한 검색

print("\n" + "="*80)
print("🔄 기존 시스템(panel_embeddings) 테스트")
print("="*80 + "\n")

# 기존 시스템 초기화
# search_system_old = PanelSearchSystemV2(config, use_chunks=False)

# 동일 쿼리로 검색
# query_compare = "경기에 사는 흡연을 하는 20대 남성"
# results_old = search_system_old.search(query_compare, top_k=5)

# print("\n기존 시스템 결과:")
# for i, mb_sn in enumerate(results_old['mb_sn_list'], 1):
#     print(f"  {i}. {mb_sn}")

# search_system_old.close()
"""

print("✅ 기존 시스템 비교 코드는 주석 처리되어 있습니다.")
print("   필요시 주석을 해제하여 panel_embeddings 기반 검색과 비교할 수 있습니다.")

## 22. 기존 시스템(panel_embeddings) vs 청킹 시스템(panel_chunks) 비교

In [ ]:
# 3가지 집계 방식 비교
print("\n" + "="*80)
print("📊 집계 방식별 Top-5 패널 비교")
print("="*80 + "\n")

query_test = "경기에 사는 흡연을 하는 20대 남성"

comparison_results = {}

for agg_method in ['max', 'mean', 'sum']:
    print(f"\n{'='*80}")
    print(f"[{agg_method.upper()}] 집계 방식")
    print("="*80)
    
    results_temp = search_system_v2.search(
        query_test, 
        top_k=5, 
        top_n_chunks=100, 
        aggregation=agg_method
    )
    
    comparison_results[agg_method] = results_temp['mb_sn_list']
    
    print("\n")

# 결과 비교 테이블
print("\n" + "="*80)
print("📋 집계 방식별 Top-5 비교표")
print("="*80 + "\n")

comparison_df = pd.DataFrame({
    'MAX': comparison_results['max'],
    'MEAN': comparison_results['mean'],
    'SUM': comparison_results['sum']
})

comparison_df.index = [f"{i+1}위" for i in range(5)]
print(comparison_df.to_string())

## 21. 집계 방식 비교 (max vs mean vs sum)

In [ ]:
# 선정된 패널의 관련 청크 내용 확인
print("\n" + "="*80)
print("📝 선정된 패널의 관련 청크 내용")
print("="*80 + "\n")

for rank, mb_sn in enumerate(results_v2['mb_sn_list'][:3], 1):
    panel_chunks = chunk_df[chunk_df['mb_sn'] == mb_sn].sort_values('similarity', ascending=False)
    
    print(f"\n{'='*80}")
    print(f"[{rank}위] {mb_sn}")
    print("="*80)
    
    for i, row in panel_chunks.head(3).iterrows():
        print(f"\n  청크 ID: {row['chunk_id']}")
        print(f"  유사도: {row['similarity']:.4f}")
        print(f"  내용: {row['chunk_text'][:200]}...")
        print("  " + "-"*76)

In [ ]:
# 청크 검색 결과 확인
print("\n" + "="*80)
print("🔍 청크 검색 결과 (상위 20개)")
print("="*80 + "\n")

chunk_df = results_v2['chunk_results']
print(chunk_df.head(20).to_string())

# 패널별 청크 분포
print("\n" + "="*80)
print("📊 패널별 청크 분포 (검색된 청크 중)")
print("="*80 + "\n")

panel_chunk_counts = chunk_df.groupby('mb_sn').agg({
    'chunk_id': 'count',
    'similarity': ['max', 'mean', 'sum']
}).round(4)

panel_chunk_counts.columns = ['청크수', '최고점수', '평균점수', '합계점수']
panel_chunk_counts = panel_chunk_counts.sort_values('최고점수', ascending=False).head(15)

print(panel_chunk_counts.to_string())

## 20. Chunk 검색 결과 분석

In [ ]:
# 테스트 쿼리 - Chunk 기반 검색
query = "경기에 사는 흡연을 하는 20대 남성 5명"

# 검색 실행 (3가지 집계 방식 비교 가능)
results_v2 = search_system_v2.search(
    query, 
    top_k=5,           # 최종 패널 5개
    top_n_chunks=100,  # 초기 청크 100개 검색
    aggregation='max'  # 'max', 'mean', 'sum' 중 선택
)

In [ ]:
# Chunk 기반 시스템 초기화
search_system_v2 = PanelSearchSystemV2(config, use_chunks=True)

## 19. Chunk 기반 검색 테스트

In [ ]:
class PanelSearchSystemV2:
    """청킹 기반 통합 패널 검색 시스템"""
    
    def __init__(self, config: Config, use_chunks: bool = True):
        self.config = config
        self.use_chunks = use_chunks
        
        print("\n🚀 시스템 초기화 중 (V2 - Chunk 기반)...\n")
        
        self.parser = QueryParser(config)
        self.textualizer = QueryTextualizer(config)
        self.embedder = EmbeddingGenerator(config.embedding_model)
        
        if use_chunks:
            self.searcher = ChunkBasedSearcher(config)
        else:
            self.searcher = DatabaseSearcher(config)
        
        print("\n✅ 시스템 초기화 완료!\n")
    
    def search(self, query: str, top_k: int = None, 
               top_n_chunks: int = 100, aggregation: str = 'max') -> Dict[str, Any]:
        """통합 검색 (청킹 기반)
        
        Args:
            query: 자연어 쿼리
            top_k: 최종 선정할 패널 개수
            top_n_chunks: 초기 검색할 청크 개수
            aggregation: 패널 점수 집계 방식 ('max', 'mean', 'sum')
        """
        print(f"\n{'='*80}")
        print(f"🔎 검색 쿼리: {query}")
        print(f"{'='*80}\n")
        
        # 1. 쿼리 파싱 (메타데이터 추출)
        print("[1단계] 쿼리 파싱 - 메타데이터 추출")
        parsed = self.parser.parse(query)
        print(f"  ✓ 지역: {parsed.get('location')}")
        print(f"  ✓ 나이: {parsed.get('age_min')}~{parsed.get('age_max')}세")
        print(f"  ✓ 성별: {parsed.get('gender')}")
        print(f"  ✓ 키워드: {parsed.get('keywords')}")
        
        # 2. 텍스트화 (프롬프트 엔지니어링)
        print("\n[2단계] 텍스트화 - 한 문장 생성 (프롬프트 엔지니어링)")
        search_text = self.textualizer.textualize(parsed)
        print(f"  ✓ 검색 텍스트: {search_text}")
        
        # 3. 임베딩
        print("\n[3단계] 임베딩 생성")
        embedding = self.embedder.embed(search_text)
        print(f"  ✓ 차원: {embedding.shape}")
        
        # 4. 청크 기반 벡터 검색
        print(f"\n[4단계] 청크 기반 벡터 검색")
        if self.use_chunks:
            chunk_df = self.searcher.search_similar_chunks(embedding, top_n_chunks)
            
            # 5. 패널별 점수 집계 및 Top-K 선정
            print(f"\n[5단계] 패널별 점수 집계 및 Top-K 선정")
            top_k = top_k or self.config.top_k
            mb_sn_list = self.searcher.aggregate_panel_scores(
                chunk_df, top_k=top_k, aggregation=aggregation
            )
            
            # 6. 전체 데이터 조회
            print(f"\n[6단계] 전체 데이터 조회")
            panel_data = self.searcher.get_panel_data(mb_sn_list)
            
            result = {
                'query': query,
                'parsed_query': parsed,
                'search_text': search_text,
                'chunk_results': chunk_df,
                'mb_sn_list': mb_sn_list,
                'panel_data': panel_data,
                'total_count': len(panel_data),
                'aggregation_method': aggregation
            }
        else:
            # 기존 방식 (panel_embeddings)
            top_k = top_k or self.config.top_k
            mb_sn_list = self.searcher.search_similar(embedding, top_k)
            panel_data = self.searcher.get_panel_data(mb_sn_list)
            
            result = {
                'query': query,
                'parsed_query': parsed,
                'search_text': search_text,
                'mb_sn_list': mb_sn_list,
                'panel_data': panel_data,
                'total_count': len(panel_data)
            }
        
        print(f"\n{'='*80}")
        print(f"✅ 검색 완료!")
        print(f"{'='*80}\n")
        
        return result
    
    def close(self):
        """시스템 종료"""
        self.searcher.close()

print("✅ PanelSearchSystemV2 정의 완료")

In [ ]:
class ChunkBasedSearcher:
    """청킹 기반 벡터 검색"""
    
    def __init__(self, config: Config):
        self.config = config
        self.conn = None
        self.connect()
    
    def connect(self):
        """DB 연결"""
        try:
            self.conn = psycopg2.connect(
                host=self.config.db_host,
                port=self.config.db_port,
                dbname=self.config.db_name,
                user=self.config.db_user,
                password=self.config.db_password
            )
            self.conn.autocommit = True
            print(f"✅ DB 연결 (Chunk 기반): {self.config.db_name}")
        except Exception as e:
            print(f"❌ DB 연결 실패: {e}")
            raise
    
    def search_similar_chunks(self, query_embedding: np.ndarray, top_n: int = 50) -> pd.DataFrame:
        """청크 기반 벡터 유사도 검색
        
        Args:
            query_embedding: 쿼리 임베딩 벡터
            top_n: 최대 추출할 청크 개수
            
        Returns:
            DataFrame: mb_sn, chunk_id, chunk_text, similarity
        """
        cursor = self.conn.cursor()
        
        # 벡터를 문자열로 변환
        vector_str = '[' + ','.join(map(str, query_embedding)) + ']'
        
        # 청크 단위 유사도 검색
        query = f"""
        SELECT 
            mb_sn,
            chunk_id,
            chunk_text,
            1 - (embedding <=> '{vector_str}'::vector) as similarity
        FROM testcl.panel_chunks
        ORDER BY similarity DESC
        LIMIT {top_n}
        """
        
        try:
            print(f"\n🔍 청크 기반 벡터 검색 (Top-{top_n})...")
            cursor.execute(query)
            results = cursor.fetchall()
            
            # DataFrame 생성
            df = pd.DataFrame(results, columns=['mb_sn', 'chunk_id', 'chunk_text', 'similarity'])
            
            print(f"✅ {len(df)}개 청크 검색 완료")
            
            cursor.close()
            return df
            
        except Exception as e:
            print(f"❌ 검색 실패: {e}")
            cursor.close()
            raise
    
    def aggregate_panel_scores(self, chunk_df: pd.DataFrame, top_k: int = 10, 
                               aggregation: str = 'max') -> List[str]:
        """패널별 점수 집계 및 Top-K 선정
        
        Args:
            chunk_df: 청크 검색 결과 DataFrame
            top_k: 최종 선정할 패널 개수
            aggregation: 'max', 'mean', 'sum' 중 선택
            
        Returns:
            List[mb_sn]: Top-K 패널 고유번호 리스트
        """
        print(f"\n📊 패널별 점수 집계 (방식: {aggregation})...")
        
        if aggregation == 'max':
            # 각 패널의 최고 점수 청크 사용
            panel_scores = chunk_df.groupby('mb_sn')['similarity'].max()
        elif aggregation == 'mean':
            # 각 패널의 평균 점수 사용
            panel_scores = chunk_df.groupby('mb_sn')['similarity'].mean()
        elif aggregation == 'sum':
            # 각 패널의 점수 합계 사용
            panel_scores = chunk_df.groupby('mb_sn')['similarity'].sum()
        else:
            raise ValueError(f"Unknown aggregation method: {aggregation}")
        
        # Top-K 선정
        top_panels = panel_scores.sort_values(ascending=False).head(top_k)
        
        print(f"✅ Top-{top_k} 패널 선정 완료\n")
        
        # 결과 출력
        for rank, (mb_sn, score) in enumerate(top_panels.items(), 1):
            chunk_count = len(chunk_df[chunk_df['mb_sn'] == mb_sn])
            print(f"  {rank:2d}. {mb_sn:30s} | 점수: {score:.4f} | 청크수: {chunk_count}")
        
        return top_panels.index.tolist()
    
    def get_panel_data(self, mb_sn_list: List[str]) -> pd.DataFrame:
        """패널 전체 데이터 조회 (DatabaseSearcher와 동일)"""
        cursor = self.conn.cursor()
        
        mb_sn_tuple = "('" + "','".join(mb_sn_list) + "')"
        
        query = f"""
        SELECT 
            w1.mb_sn,
            w1.gender as w1_gender,
            w1.birth_year,
            w1.location as w1_location,
            w1.detail_location,
            w2.data as welcome_2nd_data,
            qa.answers as quick_answers,
            qa.gender as qa_gender,
            qa.age as qa_age,
            qa.location as qa_location
        FROM "{self.config.rawdata_schema}"."welcome_1st" w1
        LEFT JOIN "{self.config.rawdata_schema}"."welcome_2nd" w2 ON w1.mb_sn = w2.mb_sn
        LEFT JOIN "{self.config.rawdata_schema}"."quick_answer" qa ON w1.mb_sn = qa.mb_sn
        WHERE w1.mb_sn IN {mb_sn_tuple}
        ORDER BY ARRAY_POSITION(ARRAY{mb_sn_list}::text[], w1.mb_sn)
        """
        
        try:
            print("\n📊 패널 전체 데이터 조회 중...")
            cursor.execute(query)
            results = cursor.fetchall()
            
            columns = [
                'mb_sn', 'gender', 'birth_year', 'location', 'detail_location',
                'welcome_2nd_data', 'quick_answers', 
                'qa_gender', 'qa_age', 'qa_location'
            ]
            
            df = pd.DataFrame(results, columns=columns)
            
            print(f"✅ {len(df)}개 패널 데이터 조회 완료")
            
            cursor.close()
            return df
            
        except Exception as e:
            print(f"❌ 데이터 조회 실패: {e}")
            cursor.close()
            raise
    
    def close(self):
        """연결 종료"""
        if self.conn:
            self.conn.close()
            print("✅ DB 연결 종료 (Chunk 기반)")

print("✅ ChunkBasedSearcher 정의 완료")

## 18. Chunk 기반 검색 시스템

In [ ]:
# Panel Chunks 테이블 구조 확인
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="ProjectDB",
    user="postgres",
    password="3961"
)

cursor = conn.cursor()

# 테이블 구조 확인
print("=" * 80)
print("Panel Chunks 테이블 구조")
print("=" * 80)

cursor.execute("""
    SELECT column_name, data_type, character_maximum_length
    FROM information_schema.columns
    WHERE table_schema = 'testcl' AND table_name = 'panel_chunks'
    ORDER BY ordinal_position;
""")

columns = cursor.fetchall()
for col in columns:
    print(f"  {col[0]:20s} | {col[1]:20s} | {col[2]}")

# 청크 개수 확인
print("\n" + "=" * 80)
print("패널별 청크 개수 (상위 10개)")
print("=" * 80)

cursor.execute("""
    SELECT mb_sn, COUNT(*) as chunk_count
    FROM testcl.panel_chunks
    GROUP BY mb_sn
    ORDER BY chunk_count DESC
    LIMIT 10;
""")

chunk_counts = cursor.fetchall()
for mb_sn, count in chunk_counts:
    print(f"  {mb_sn:30s} | {count:3d} chunks")

# 샘플 데이터 확인
print("\n" + "=" * 80)
print("샘플 데이터 (1개 패널의 청크들)")
print("=" * 80)

cursor.execute("""
    SELECT mb_sn, chunk_id, chunk_text, pg_typeof(embedding) as embedding_type
    FROM testcl.panel_chunks
    LIMIT 3;
""")

samples = cursor.fetchall()
for i, sample in enumerate(samples, 1):
    print(f"\n청크 {i}:")
    print(f"  mb_sn: {sample[0]}")
    print(f"  chunk_id: {sample[1]}")
    print(f"  chunk_text: {sample[2][:100]}...")
    print(f"  embedding_type: {sample[3]}")

cursor.close()
conn.close()

print("\n✅ 확인 완료")

## 17. Panel Chunks 테이블 구조 확인